<a href="https://colab.research.google.com/github/uchoagab/Desafio-VLab/blob/load_dataset/Desafio2_VLab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Carregamento e Limpeza do dataset

**Importação de todos os componentes necessários para a análise. O dataset "GoodReads 100k books" (https://www.kaggle.com/datasets/mdhamani/goodreads-books-100k) foi baixado localmente e subido para o Colab.**

In [52]:
import pandas as pd
import numpy as np

In [53]:
df = pd.read_csv('sample_data/GoodReads_100k_books.csv')

In [54]:
df.info()
df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 13 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   author        100000 non-null  object 
 1   bookformat    96772 non-null   object 
 2   desc          93228 non-null   object 
 3   genre         89533 non-null   object 
 4   img           96955 non-null   object 
 5   isbn          85518 non-null   object 
 6   isbn13        88565 non-null   object 
 7   link          100000 non-null  object 
 8   pages         100000 non-null  int64  
 9   rating        100000 non-null  float64
 10  reviews       100000 non-null  int64  
 11  title         99999 non-null   object 
 12  totalratings  100000 non-null  int64  
dtypes: float64(1), int64(3), object(9)
memory usage: 9.9+ MB


,pages,rating,reviews,totalratings
count,100000.000000,100000.000000,100000.000000,1.000000e+05
mean,255.010240,3.833055,181.528450,2.990764e+03
std,367.913582,0.621237,1449.451229,3.635338e+04
min,0.000000,0.000000,0.000000,0.000000e+00
25%,135.000000,3.660000,3.000000,3.100000e+01
50%,240.000000,3.910000,15.000000,1.460000e+02
75%,336.000000,4.140000,67.000000,7.440000e+02
max,70000.000000,5.000000,158776.000000,3.819326e+06


In [55]:
# Remoção das colunas irrelevantes
df.drop(columns=['isbn', 'isbn13', 'link'], axis=1, inplace=True)

In [56]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 10 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   author        100000 non-null  object 
 1   bookformat    96772 non-null   object 
 2   desc          93228 non-null   object 
 3   genre         89533 non-null   object 
 4   img           96955 non-null   object 
 5   pages         100000 non-null  int64  
 6   rating        100000 non-null  float64
 7   reviews       100000 non-null  int64  
 8   title         99999 non-null   object 
 9   totalratings  100000 non-null  int64  
dtypes: float64(1), int64(3), object(6)
memory usage: 7.6+ MB


In [57]:
df = df.dropna(subset=['rating', 'totalratings'])

In [58]:
# Agregar por [autor, título], remover duplicatas criando nova avaliação
grouped_df = df.groupby(['author', 'title'], as_index=False).agg({
    'bookformat': 'first',
    'desc': 'first',
    'genre': 'first',
    'img': 'first',
    'pages': 'first',
    'reviews': 'sum',
    'totalratings': 'sum',
    'rating': lambda x: (x * df.loc[x.index, 'totalratings']).sum() / df.loc[x.index, 'totalratings'].sum()  # Média ponderada das avaliações duplicadas
})

df = grouped_df

<ipython-input-58-2e2bdd237ccd>:10: RuntimeWarning: invalid value encountered in scalar divide
  'rating': lambda x: (x * df.loc[x.index, 'totalratings']).sum() / df.loc[x.index, 'totalratings'].sum()  # Média ponderada das avaliações duplicadas


In [59]:
# Checar existência de valores NaN por coluna
missing_percentage = df.isnull().mean() * 100
print(missing_percentage)

author           0.000000
title            0.000000
bookformat       3.177955
desc             6.728257
genre           10.470738
img              3.042830
pages            0.000000
reviews          0.000000
totalratings     0.000000
rating           1.559450
dtype: float64


**Como a única coluna onde mais de 10% dos valores são NaN é gênero, podemos converter seus elementos vazios para string vazia e depois transformar todos em listas.**

---



In [60]:
print(df['genre'][1])

History,European Literature,German Literature,Reference,Dictionaries


In [61]:
# Trocar NaN por string vazia
df['genre'] = df['genre'].fillna('')

# Converter elementos da coluna de gênero por lista, divindo as string por vírgula
df['genre'] = df['genre'].apply(lambda x: [genre.strip() for genre in x.split(',')] if x else [])

In [62]:
print(df['genre'][1])

['History', 'European Literature', 'German Literature', 'Reference', 'Dictionaries']


**Assim, os dados estão prontos para serem usados.**


---



